In [ ]:
from scipy.spatial import distance
from imutils import face_utils
# from pydub import AudioSegment
# from pydub.playback import play
import numpy as np
import _thread
import imutils
import time
import dlib
import cv2

def eye_aspect_ratio(eye):
    # calcular las distancias euclidianas entre los dos conjuntos de
    # landmarks (coordenadas x, y) verticales de los ojos
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    # calcular la distancia euclidiana entre los landmarks
    # horizontales de los ojos (coordenadas x, y)
    C = distance.euclidean(eye[0], eye[3])
    
    # calcular la relación de aspecto del ojo
    ear = (A + B) / (2.0 * C)
    
    # retornar la relación de aspecto del ojo
    return ear

hilo_flag = True  # Solo para desarrollo, en la versión final debería ser True
l_alarma = _thread.allocate_lock()  # Puede haber N Locks, pero solo se puede adquirir uno a la vez

# def sound_alarm():
#     global ejecucion
#     while hilo_flag:
#         l_alarma.acquire()
#         ALARM = AudioSegment.from_mp3("sound/alarm3.mp3")
#         for i in np.arange(3):
#             play(ALARM)

# definir dos constantes, una para la relación de aspecto del ojo para indicar
# el parpadeo y luego una segunda constante para el número de fotogramas consecutivos
# que el ojo debe estar por debajo del umbral para activar la alarma
EYE_AR_THRESH = 0.22
EYE_AR_CONSEC_FRAMES = 10
EYE_AR_NOT_DETECTED_FRAMES = 20

# inicializar los contadores de fotogramas para somnolencia y distracción de ojos
# así como un booleano utilizado para indicar si la alarma está sonando
COUNTER_DROWSINESS = 0
COUNTER_EYES_NOT_DETECTED = 0
ALARM_ON = False
ENVIO_ALERTA = False
MENSAJE_ALERTA = ""

# inicializar el detector de caras de dlib (basado en HOG) y luego crear
# el predictor de landmarks faciales
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # El archivo Dat es el núcleo del código

# obtener los índices de los landmarks faciales para el ojo izquierdo y derecho, respectivamente
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# l_alarma.acquire()  # Obtener el candado antes de crear el hilo
# _thread.start_new_thread(sound_alarm, ())  # Crea un único hilo para las alarmas

def deteccionSomnolencia(frame):
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)

    global COUNTER_DROWSINESS 
    global COUNTER_EYES_NOT_DETECTED
    
    frame = imutils.resize(frame, width=450)
    
    # detectar caras en el fotograma en escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    
    # si no se detectan caras en el fotograma
    if not subjects:
        COUNTER_EYES_NOT_DETECTED += 1        
        if COUNTER_EYES_NOT_DETECTED >= EYE_AR_NOT_DETECTED_FRAMES:
            # Si está distraído
            MENSAJE_ALERTA = "Conductor distraído : " + current_time 
            cv2.putText(frame, MENSAJE_ALERTA, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            if l_alarma.locked():
                l_alarma.release()
    else:
        COUNTER_EYES_NOT_DETECTED = 0
        time.sleep(0.02)
        if not l_alarma.locked():
            l_alarma.acquire()

        for subject in subjects:
            # determinar los landmarks faciales para la región facial, luego
            # convertir los landmarks faciales (coordenadas x, y) en un array NumPy
            shape = predict(gray, subject)
            shape = face_utils.shape_to_np(shape)  # convirtiendo a un array NumPy

            # extraer las coordenadas del ojo izquierdo y derecho, luego calcular la
            # relación de aspecto del ojo para ambos ojos
            leftEye = shape[lStart:lEnd]           
            rightEye = shape[rStart:rEnd]

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)

            # promediar la relación de aspecto del ojo para ambos ojos
            ear = (leftEAR + rightEAR) / 2.0

            # calcular el casco convexo para el ojo izquierdo y derecho
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
        
            # visualizar cada uno de los ojos
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
            # verificar si la relación de aspecto del ojo está por debajo del umbral de parpadeo
            # y si es así, incrementar el contador de fotogramas de parpadeo
            if ear < EYE_AR_THRESH:      
                COUNTER_DROWSINESS += 1
                # si los ojos estuvieron cerrados durante un número suficiente de fotogramas, entonces
                # sonar la alarma
                if COUNTER_DROWSINESS >= EYE_AR_CONSEC_FRAMES:
                        # Si está somnoliento
                        MENSAJE_ALERTA = "Conductor dormido : " + current_time  
                        cv2.putText(frame, MENSAJE_ALERTA, (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                        if l_alarma.locked():
                            l_alarma.release()
            # de lo contrario, la relación de aspecto del ojo no está por debajo del umbral de parpadeo,
            # por lo que se reinicia el contador y la alarma
            else:
                COUNTER_DROWSINESS = 0
                time.sleep(0.02)
                if not l_alarma.locked():
                    l_alarma.acquire()
                    
    return frame


In [ ]:
from scipy.spatial import distance
from imutils import face_utils
import _thread  # Agregar esta línea para importar el módulo _thread
import numpy as np
import imutils
import time
import dlib
import cv2

def eye_aspect_ratio(eye):
    # calcular las distancias euclidianas entre los dos conjuntos de
    # landmarks (coordenadas x, y) verticales de los ojos
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    # calcular la distancia euclidiana entre los landmarks
    # horizontales de los ojos (coordenadas x, y)
    C = distance.euclidean(eye[0], eye[3])
    
    # calcular la relación de aspecto del ojo
    ear = (A + B) / (2.0 * C)
    
    # retornar la relación de aspecto del ojo
    return ear

hilo_flag = True  # Solo para desarrollo, en la versión final debería ser True
l_alarma = _thread.allocate_lock()  # Puede haber N Locks, pero solo se puede adquirir uno a la vez

# definir dos constantes, una para la relación de aspecto del ojo para indicar
# el parpadeo y luego una segunda constante para el número de fotogramas consecutivos
# que el ojo debe estar por debajo del umbral para activar la alarma
EYE_AR_THRESH = 0.22
EYE_AR_CONSEC_FRAMES = 10
EYE_AR_NOT_DETECTED_FRAMES = 20

# inicializar los contadores de fotogramas para somnolencia y distracción de ojos
# así como un booleano utilizado para indicar si la alarma está sonando
COUNTER_DROWSINESS = 0
COUNTER_EYES_NOT_DETECTED = 0
ALARM_ON = False
ENVIO_ALERTA = False
MENSAJE_ALERTA = ""

# inicializar el detector de caras de dlib (basado en HOG) y luego crear
# el predictor de landmarks faciales
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # El archivo Dat es el núcleo del código

# obtener los índices de los landmarks faciales para el ojo izquierdo y derecho, respectivamente
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

l_alarma.acquire()  # Obtener el candado antes de crear el hilo
#_thread.start_new_thread(sound_alarm, ())  # Crea un único hilo para las alarmas

def deteccionSomnolencia(frame):
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)

    global COUNTER_DROWSINESS 
    global COUNTER_EYES_NOT_DETECTED
    
    frame = imutils.resize(frame, width=450)
    
    # detectar caras en el fotograma en escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    
    # si no se detectan caras en el fotograma
    if not subjects:
        COUNTER_EYES_NOT_DETECTED += 1        
        if COUNTER_EYES_NOT_DETECTED >= EYE_AR_NOT_DETECTED_FRAMES:
            # Si está distraído
            MENSAJE_ALERTA = "Conductor distraído : " + current_time 
            cv2.putText(frame, MENSAJE_ALERTA, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            if l_alarma.locked():
                l_alarma.release()
    else:
        COUNTER_EYES_NOT_DETECTED = 0
        time.sleep(0.02)
        if not l_alarma.locked():
            l_alarma.acquire()

        for subject in subjects:
            # determinar los landmarks faciales para la región facial, luego
            # convertir los landmarks faciales (coordenadas x, y) en un array NumPy
            shape = predict(gray, subject)
            shape = face_utils.shape_to_np(shape)  # convirtiendo a un array NumPy

            # extraer las coordenadas del ojo izquierdo y derecho, luego calcular la
            # relación de aspecto del ojo para ambos ojos
            leftEye = shape[lStart:lEnd]           
            rightEye = shape[rStart:rEnd]

            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)

            # promediar la relación de aspecto del ojo para ambos ojos
            ear = (leftEAR + rightEAR) / 2.0

            # calcular el casco convexo para el ojo izquierdo y derecho
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
        
            # visualizar cada uno de los ojos
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
            # verificar si la relación de aspecto del ojo está por debajo del umbral de parpadeo
            # y si es así, incrementar el contador de fotogramas de parpadeo
            if ear < EYE_AR_THRESH:      
                COUNTER_DROWSINESS += 1
                # si los ojos estuvieron cerrados durante un número suficiente de fotogramas, entonces
                # sonar la alarma
                if COUNTER_DROWSINESS >= EYE_AR_CONSEC_FRAMES:
                        # Si está somnoliento
                        MENSAJE_ALERTA = "Conductor dormido : " + current_time  
                        cv2.putText(frame, MENSAJE_ALERTA, (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                        if l_alarma.locked():
                            l_alarma.release()
            # de lo contrario, la relación de aspecto del ojo no está por debajo del umbral de parpadeo,
            # por lo que se reinicia el contador y la alarma
            else:
                COUNTER_DROWSINESS = 0
                time.sleep(0.02)
                if not l_alarma.locked():
                    l_alarma.acquire()
                    
    return frame

# Capturar video desde la cámara
video_capture = cv2.VideoCapture(0)

while True:
    # Capturar fotograma por fotograma
    ret, frame = video_capture.read()

    # Pasar el fotograma a la función de detección de somnolencia
    frame = deteccionSomnolencia(frame)

    # Mostrar el fotograma resultante
    cv2.imshow('Video', frame)

    # Detener el bucle si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura de video y cerrar la ventana
video_capture.release()
cv2.destroyAllWindows()

